In [ ]:
import tensorflow as tf
import gym

from rltensor.agents import DQN
from rltensor.processors import AtariProcessor
from rltensor.networks import DuelingModel


conf = {"q_conf":[
            {"name": "conv2d", "kernel_size":(8, 8), "num_filter":32, "stride":4,
             "padding": 'SAME', "is_batch":False, 'activation': tf.nn.relu},
            {"name": "conv2d", "kernel_size":(5, 5), "num_filter":64, "stride":2,
             "padding": 'SAME', "is_batch":True, 'activation': tf.nn.relu},
           {"name": "conv2d", "kernel_size": (3, 3), "num_filter":64, "stride":1,
             "padding": 'SAME', "is_batch":True, 'activation': tf.nn.relu},
            {"name": "dense", "is_flatten":True, "is_batch":True, "num_hidden": 512, 'activation': tf.nn.relu},
        ],
        'double_q': True,
        "memory_limit": 100000,
        "window_length": 4,
        "gamma": 0.99,
        "learning_rate": 2.5e-4,
        "learning_rate_minimum": 2.5e-4,
        "learning_rate_decay": 0.9,
        "learning_rate_decay_step": 100,
        "ep": 1e-3,
        "min_r": -1,
        "max_r": 1,
        "batch_size": 32,
        "error_clip": 1.0,
        "processor": AtariProcessor(84, 84),
        "t_learn_start": 100,
        "t_train_freq": 4,
        "t_target_q_update_freq": 10000,
        "ep_start": 1.0,
        "ep_end": 0.1,
        "t_ep_end": int(1e6),
        "model_dir": "./logs/dqn",
        "log_freq": 1000,
        "avg_length": 10000,
        "env_name": 'DemonAttack-v0',
        "prioritized": True,
}

conf = {"q_conf":[
            {"name": "conv2d", "kernel_size":(8, 8), "num_filter":32, "stride":4,
             "padding": 'SAME', "is_batch":False, 'activation': tf.nn.relu},
            {"name": "conv2d", "kernel_size":(5, 5), "num_filter":64, "stride":2,
             "padding": 'SAME', "is_batch":True, 'activation': tf.nn.relu},
           {"name": "conv2d", "kernel_size": (3, 3), "num_filter":64, "stride":1,
             "padding": 'SAME', "is_batch":True, 'activation': tf.nn.relu},
            {"name": "dense", "is_flatten":True, "is_batch":True, "num_hidden": 512, 'activation': tf.nn.relu},
        ],
        "env_name": 'DemonAttack-v0',
        "processor": AtariProcessor(84, 84),
}

env = gym.make('DemonAttack-v0')
tf.reset_default_graph()
dqn = DQN(env, conf, q_network_cls=DuelingModel)
dqn.train(int(1e7), render_freq=None, save_video_path="./videos")

[2017-07-21 01:33:56,025] Making new env: DemonAttack-v0
[2017-07-21 01:33:57,266] Finished writing results. You can upload them to the scoreboard via gym.upload('/home/tomoaki/work/Development/RL/videos')
[2017-07-21 01:33:57,564] Clearing 4 monitor files from previous run (because force=True was provided)
[2017-07-21 01:33:57,579] Starting new video recorder writing to /home/tomoaki/work/Development/RL/videos/openaigym.video.2.19738.video000000.mp4


  0%|          | 0/10000000 [00:00<?, ?it/s]

Model saved in file: params/model.ckpt



  0%|          | 58/10000000 [00:00<4:52:41, 569.42it/s]/home/tomoaki/anaconda3/lib/python3.6/site-packages/rltensor-0.1.0-py3.6.egg/rltensor/memories/prioritized.py:31: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.

  0%|          | 519/10000000 [00:08<54:37:38, 50.85it/s][2017-07-21 01:34:06,292] Starting new video recorder writing to /home/tomoaki/work/Development/RL/videos/openaigym.video.2.19738.video000001.mp4
/home/tomoaki/anaconda3/lib/python3.6/site-packages/rltensor-0.1.0-py3.6.egg/rltensor/memories/prioritized.py:31: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  0%|          | 7096/10000000 [02:18<53:42:56, 51.68it/s]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.hist(dqn.memory.priorities, bins=100)

In [2]:
"dopuble_q" not in {}

True

In [ ]:
if [1, 2, 3]:
    print("hello")

In [ ]:
env.action_space.n

In [ ]:
y.get_shape().as_list()

In [ ]:
count = 4
while count < 5:
    print(count)
    count += 1

In [ ]:
"%s" % True

In [ ]:
a.insert(0, 2)

In [15]:
a

[2, 1]

In [11]:
np.random.randint(0, 2, 10)

array([0, 0, 0, 1, 0, 0, 0, 1, 0, 0])

In [42]:
from collections import deque

In [44]:
x = deque([1, 2, 3], maxlen=5)

In [45]:
x.append(3)
x.append(3)
x.append(3)

In [46]:
x

deque([2, 3, 3, 3, 3])

In [57]:
 result = tf.select(pred, val_if_true, val_if_false)

AttributeError: module 'tensorflow' has no attribute 'select'

In [60]:
x = tf.placeholder(tf.bool, (None,))
y = tf.cast(x, tf.int32)
z = tf.one_hot(y, 2)

In [62]:
sess = tf.InteractiveSession()
print(y.eval(feed_dict={x:[True, False, True]}))
print(z.eval(feed_dict={x:[True, False, True]}))

[1 0 1]
[[ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]]


In [13]:
type(np.arange(10).astype(int)[0])

numpy.int64

In [16]:
type(np.random.choice(range(0, 5), 3)[0])

numpy.int64

In [44]:
np.random.choice([1, 2, 3, 4], 3, False)

array([1, 4, 3])

In [12]:
x = np.arange(10)
np.append(x, 10)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [13]:
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])